In [1]:
import pandas as pd

In [101]:
import re

## prepare data

In [540]:
rel_train = pd.read_csv('relevance_train.csv')
rel_test = pd.read_csv('relevance_test.csv')

In [545]:
tr_q_id = []
tr_d_id = []
tr_rel = []

with open('relevance_train.csv') as file:
    lines = file.readlines()
    for l in lines:
        if '\t' not in l:
            continue
            
        spl = l.split()
        tr_q_id.append(spl[0])
        tr_d_id.append(spl[1])
        tr_rel.append(spl[2])

In [496]:
regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string
regex.sub('', 'ab3d*E')

'abdE'

In [502]:
queries = {}
with open('queries.csv') as file:
    lines = file.readlines()
    for line in lines:
        if '\t' in line:
            splits = line.split('\t')
            t = regex.sub(' ', splits[1])
            queries[splits[0]] = clean_query(t)

In [503]:
article_start = 0
article_lines = []

query2text = {}

lines = []
with open('Documents.csv') as docs:
    lines = docs.readlines()

for i,line in enumerate(lines):
    if re.match('.*Id.*\d', line) is not None and article_start != i:
        article_lines = lines[article_start:i]
        article_start = i
        q_id = line.split()[1]
        query2text[q_id] = parse_article_lines(article_lines)

In [293]:
from functools import reduce
import nltk

In [294]:
ps = nltk.stem.snowball.SnowballStemmer('english')

In [295]:
import stop_words

In [296]:
sw = stop_words.get_stop_words('english')

In [501]:
def clean_query(tq):
    result = " "
    for w in tq.split():
        if w not in sw:
            result += ps.stem(w.replace(r'[^a-zA-Z]', '')) + " "
    return result

In [500]:
def parse_article_lines(lines):
    text_index = 0
    for i,l in enumerate(lines):
        if '.W' in l:
            text_index = i+1
    
    text_lines = lines[text_index:len(lines)]
    text_lines = list(map(lambda l: regex.sub(' ',l), text_lines))
    text_lines = list(map(clean_query, text_lines))
    return text_lines

In [475]:
def flat(list_of_list):
    result = []
    for li in list_of_list:
        for el in li:
            result.append(el)
            
    result = list(filter(lambda l:len(line) > 1, result))
    
    return result

In [504]:
for k in query2text.keys():
    query2text[k] = flat(list(map(lambda l: l.split(), query2text[k])))

In [505]:
for k in query2text.keys():
    query2text[k] = list(filter(lambda l: len(l) > 2, query2text[k]))

In [506]:
for k in queries.keys():
    queries[k] = queries[k].split()

In [507]:
for k in queries.keys():
    queries[k] = list(filter(lambda l:len(line) > 2, queries[k]))

In [121]:
import gensim

In [535]:
all_w = list(query2text.values())
all_w.append(queries.values())



model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

## now prepare model

In [353]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [513]:
model = gensim.models.Word2Vec(min_count=1,workers=4, hs=1, negative=0)

In [514]:
model.build_vocab(flat(all_w))

In [515]:
kw = model.wv

In [523]:
import numpy as np

## fix that, i suppose should be some library method or somehow fix vocaulary

In [528]:
def get_sen_vector(sen):
    v = np.zeros(100)
    for w in sen:
        try:
            word_v = kw[w]
            v = np.add(v, word_v)
        except KeyError as e:
            pass
        
    return v

In [521]:
len(kw['test'])

100

## prepare now pairs doc - request

In [585]:
vectors = []
labels = []

for i in range(1,len(tr_d_id)):
    if tr_d_id[i] in query2text and tr_q_id[i] in queries:
        labels.append(int(tr_rel[i]))
        q_v = get_sen_vector(queries[tr_q_id[i]])
        d_v = get_sen_vector(query2text[tr_d_id[i]])
        vectors.append(np.hstack([d_v, q_v]))
        
vectors = np.array(vectors)
labels = np.array(labels).reshape(len(labels),1)

## prepare classification model

In [571]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [586]:
trainX, testX, trainY, testY = train_test_split(vectors, labels)

In [587]:
dtrain = xgb.DMatrix(trainX, trainY)
dtest = xgb.DMatrix(testX, testY)

In [594]:

param = {'silent':1, 'objective':'reg:linear' }
param['nthread'] = 8
param['eval_metric'] = 'rmse'
param['eta'] = 0.075
param['max_depth'] = 12
param['silent'] = 1
param['n_estimators'] = 400
param['early_stopping_rounds'] = 30

watchlist = [ (dtrain,'train'), (dtest, 'test') ]

In [595]:
bst = xgb.train(param, dtrain, 1500, watchlist,verbose_eval=25)

[0]	train-rmse:2.19522	test-rmse:2.3074
[25]	train-rmse:0.418375	test-rmse:1.59439
[50]	train-rmse:0.088585	test-rmse:1.5616
[75]	train-rmse:0.021808	test-rmse:1.55501
[100]	train-rmse:0.006261	test-rmse:1.55418
[125]	train-rmse:0.002079	test-rmse:1.55376
[150]	train-rmse:0.000752	test-rmse:1.55354
[175]	train-rmse:0.000518	test-rmse:1.5535
[200]	train-rmse:0.000518	test-rmse:1.5535
[225]	train-rmse:0.000518	test-rmse:1.5535
[250]	train-rmse:0.000518	test-rmse:1.5535
[275]	train-rmse:0.000518	test-rmse:1.5535
[300]	train-rmse:0.000518	test-rmse:1.5535
[325]	train-rmse:0.000518	test-rmse:1.5535
[350]	train-rmse:0.000518	test-rmse:1.5535
[375]	train-rmse:0.000518	test-rmse:1.5535
[400]	train-rmse:0.000518	test-rmse:1.5535
[425]	train-rmse:0.000518	test-rmse:1.5535
[450]	train-rmse:0.000518	test-rmse:1.5535
[475]	train-rmse:0.000518	test-rmse:1.5535
[500]	train-rmse:0.000518	test-rmse:1.5535
[525]	train-rmse:0.000518	test-rmse:1.5535
[550]	train-rmse:0.000518	test-rmse:1.5535
[575]	train-

In [624]:
result = []
test_q = []
test_d = []
for i,row in rel_test.iterrows():
    q_id = str(row['QueryId'])
    d_id = str(row['DocumentId'])

    test_q.append(q_id)
    test_d.append(d_id)
    
    if q_id in queries and d_id in query2text:
        d_vec = get_sen_vector(query2text[d_id])
        q_vec = get_sen_vector(queries[q_id])
        vec = np.hstack([d_vec, q_vec]).reshape(1,200)
        m = xgb.DMatrix(vec)
        pred = bst.predict(m)
        result.append(pred)
    else:
        pass
        #print('no doc at',q_id, d_id)

In [637]:
tup = zip((result, test_q, test_d))

In [644]:
output = pd.DataFrame(sorted(zip(result, test_q, test_d), key=lambda e:e[0]))

In [663]:
groupOut = output.groupby(1, as_index=False).count()

In [666]:
output

,0,1,2
0,-0.736579,161,996
1,-0.430663,127,982
2,-0.285791,175,941
3,-0.034289,131,1016
4,0.033303,180,620
5,0.044976,157,1304
6,0.159862,191,948
7,0.206981,159,1115
8,0.264965,140,954
9,0.349755,140,1041


In [670]:
output.to_csv(path_or_buf='pred.csv', columns=(1,2), header=('QueryId','DocumentId'), index=False)